In [1]:
from langchain_community.agent_toolkits.openapi.toolkit import RequestsToolkit
from langchain_community.utilities.requests import TextRequestsWrapper
toolkit = RequestsToolkit(
    requests_wrapper=TextRequestsWrapper(headers={}),
    allow_dangerous_requests=True,
)

In [2]:
for tool in toolkit.get_tools():
    print(tool.name)

requests_get
requests_post
requests_patch
requests_put
requests_delete


In [15]:
api_spec = """
openapi: 3.0.0
info:
 title: Frankfurter Currency Exchange API
 version: v1
 description: API for retrieving currency exchange rates. Pay attention to
the base currency and change it if needed.
servers:
 - url: https://api.frankfurter.dev/v1
paths:
 /v1/latest:
 get:
 summary: Get the latest exchange rates.
 parameters:
 - in: query
 name: symbols
 schema:
 type: string
 description: Comma-separated list of currency symbols to retrieve
rates for. Example: CHF,GBP
 - in: query
 name: base
 schema:
  type: string
 description: The base currency for the exchange rates. If not
provided, EUR is used as a base currency. Example: USD
 /v1/date:
 get:
 summary: Get exchange rates for a specific date.
 parameters:
 - in: path
 name: date
 schema:
 type: string
 description: The date to retrieve exchange rates for. Format: YYYY-MM-DD
"""

In [16]:
from langchain_core.prompts import PromptTemplate

system_message = PromptTemplate(
    template="You're given the API spec:\n{api_spec}. \nUse the API to answer users' queries if possible.",
    input_variables=["api_spec"]
)


In [1]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [2]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
class Plan(BaseModel):
    """Plan to follow in future"""
    steps: list[str] = Field(
        description="different steps to follow, should be in sorted order"
    )

In [4]:
system_prompt_template = (
 "For the given task, come up with a step by step plan.\n"
 "This plan should involve individual tasks, that if executed correctly will "
 "yield the correct answer. Do not add any superfluous steps.\n"
 "The result of the final step should be the final answer. Make sure that each "
 "step has all the information needed - do not skip steps."
)

In [ ]:
planner_prompt = ChatPromptTemplate.from_messages([("system", system_prompt_template), ("user", "Prepare a plan how to solve the following task:\n{task}\n")])
